# "Dynamic display of audio spectrogra of voice"
> "Wanted a simple way to show the differenct frequencies we use when we talk"


- toc: true
- branch: master
- badges: false
- comments: true
- categories: [audio, dsp, spectrogram, librosa]
- image: images/spectrogram/Feb-05-2021_12-10-51_sample_audio.gif
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

#Problem

Wanted a simple way to show that when we talk we use different frequencies.

#Solution



*   Record sound using the computers microphone
*   Analyze the audio with [librosa](https://librosa.org/)
*   Display the results dynamically



## Record sound

install libraries

In [5]:
#hide_show
!pip install pyaudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp36-cp36m-linux_x86_64.whl size=51612 sha256=acf9d048ae98e47a7f527e792f054bba54872e086016a17c761493dd93f31357
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built pyaudio


import libraries

In [4]:
#hide_show

import pyaudio
import time
import librosa
import librosa.display

import matplotlib.pyplot as plt
import numpy as np
from IPython import display

function for displaying spectrogram of audio

In [6]:
def plot_spectrogram(wave_data, sr=44100, DIV=4, n_fft=2048, hop_length=512):

    plt.clf()
    D = np.abs(librosa.stft(wave_data, n_fft=n_fft,  hop_length=hop_length))
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='linear')

    display.display(plt.gcf())
    display.clear_output(wait=True)

Code below works directly on a pc in a jupyter notebook,(NOT on a virtual machine. This wont work in colab)

Continuously stream audio to spectrogram for analysis

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
p = pyaudio.PyAudio()

frames = []
max_frames = 50
RATE = 44100
BUFFER_SIZE = 1024 * 2

def callback(in_data, frame_count, time_info, status):
    frames.append(np.frombuffer(in_data, dtype=np.int16))
    if len(frames) > max_frames:
        frames.pop(0)
    return (in_data, pyaudio.paContinue)

start_t = time.time() 

stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=RATE,
                input=True,
                frames_per_buffer=BUFFER_SIZE,
                stream_callback=callback)

stream.start_stream()

while True:
    try:
        
        if len(frames) >= 4:
            
            wave_data = np.concatenate( tuple(frames[-20:]) )
            wave_data=wave_data/9000.

            plot_spectrogram(wave_data)
        
        time.sleep(0.01)
                
    except KeyboardInterrupt:
        break

print("stop stream")
stream.stop_stream()

stream.close()
p.terminate()

# example of raw data being captured and analyzed.

![hello world](https://raw.githubusercontent.com/patrick-hanley/thought_experiments/master/images/spectrogram/Feb-05-2021_12-10-51_sample_audio.gif)